In [ ]:
import numpy as np
import torch
from stork.constraints import MinMaxConstraint
from LPLDataset import SparseGenerator, LPLDataset
from LPLModel import LPLModel
from LPLGroup import LPLGroup, InputGroup
from LPLConnection import SymmetricSTDPConnection, LPLConnection, Connection

In [ ]:
# Model Parameters
dt = 2e-3
time_window = 0.5
nb_time_steps = (int)(time_window/dt)
timestep_rmsprop_updates = nb_time_steps
batch_size = 1
nb_inputs = 500  
nb_exc_units = 100
nb_inh_units = 25
device = torch.device("cuda")
dtype = torch.float
epoch = 1
file_name = "_50000s_"

In [ ]:
data1 = np.loadtxt("spks/5hz_spikes_50000s_2.ras")
dataset1 = LPLDataset(data1, time_step=dt, time_window=time_window, device=device)
data2 = np.loadtxt("spks/5hz_spikes_100s_2.ras")
dataset2 = LPLDataset(data2, time_step=dt, time_window=time_window, device=device)

In [ ]:
lpl_con_prob = 0.1
ii_con_prob  = 0.1
ie_con_prob  = 0.5
lpl_con_w    = 0.15
ii_con_w     = 0.4
ie_con_w     = 0.5

ei_con_w     = 0.5

## LPL Model

In [ ]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

lplcon = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    constraints=constraints1,
    lamda_=1.0,
    phi=1.0,
    lr=1e-2,
    delta_time=20e-3,
    tau_rms=100.0,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

conei = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True,
        initial_weight=ei_con_w, 
        name="Con1", 
        target="exc"))

conii = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Con2", 
        target="inh"))

conie = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="STDPCon", 
        target="inh",
        constraints=constraints2,
        lr=1e-3,
        kappa=10))

generator = SparseGenerator(nb_workers=0)
model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

exc_neurons.random_mem(mean=-60e-3, sigma=10e-3, seed=42)
lplcon.set_evolve(True)
conie.set_evolve(True)


In [ ]:

model.stimulate(dataset1, epoch=epoch)

In [ ]:
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [lplcon.name]
model.configure_stored(list_neurons, list_con, file_name + "LPL")
model.stimulate(dataset2)

### Pred Off Model

In [ ]:

model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

lplcon = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    tau_var=30,
    constraints=constraints1,
    lamda_=1.0,
    phi=0.0,
    lr=1e-2,
    delta_time=20e-3,
    tau_rms=100.0,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

conei = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True,
        initial_weight=ei_con_w, 
        name="Conei", 
        target="exc"))

conii = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Conii", 
        target="inh"))

conie = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="conie", 
        target="inh",
        constraints=constraints2,
        lr=1e-3,
        kappa=10))

generator = SparseGenerator(nb_workers=0)
model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

exc_neurons.random_mem(mean=-60e-3, sigma=10e-3, seed=42)
lplcon.set_evolve(True)
conie.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)

In [ ]:
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [lplcon.name]
model.configure_stored(list_neurons, list_con, file_name + "Pred_off")
model.stimulate(dataset2)

## Hebb Off Model

In [ ]:

model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)
constraints2 = MinMaxConstraint(0.0, 10.0)

lplcon = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    tau_var=30,
    constraints=constraints1,
    lamda_=0.0,
    phi=1.0,
    lr=1e-2,
    delta_time=20e-3,
    tau_rms=100.0,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

conei = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True,
        initial_weight=ei_con_w,  
        name="Conei", 
        target="exc"))

conii = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Conii", 
        target="inh"))

conie = model.add_connection(
    SymmetricSTDPConnection(
        inh_neurons, 
        exc_neurons, 
        connection_prob=ie_con_prob, 
        initial_weight=ie_con_w, 
        name="conie", 
        target="inh",
        constraints=constraints2,
        lr=1e-3,
        kappa=10))

generator = SparseGenerator(nb_workers=0)
model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()
exc_neurons.random_mem(mean=-60e-3, sigma=10e-3, seed=42)

lplcon.set_evolve(True)
conie.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)

In [ ]:
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [lplcon.name]
model.configure_stored(list_neurons, list_con, file_name + "Hebb_off")
model.stimulate(dataset2)

## Inhib off Model

In [ ]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))

constraints1 = MinMaxConstraint(0.0, 1.0)

lplcon = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    tau_var=30,
    constraints=constraints1,
    lamda_=1.0,
    phi=1.0,
    lr=1e-2,
    delta_time=20e-3,
    tau_rms=100.0,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

generator = SparseGenerator(nb_workers=0)
model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()
exc_neurons.random_mem(mean=-60e-3, sigma=10e-3, seed=42)

lplcon.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)

In [ ]:
list_neurons = [input_group.name, exc_neurons.name]
list_con = [lplcon.name]
model.configure_stored(list_neurons, list_con, file_name + "Inhib_off")
model.stimulate(dataset2)

## Inhib Fixed Model

In [ ]:
model = LPLModel(batch_size, nb_time_steps, nb_inputs, device, dtype)

input_group = model.add_group(InputGroup(nb_inputs, name="input"))
exc_neurons = model.add_group(LPLGroup(nb_exc_units, name="exc"))
inh_neurons = model.add_group(LPLGroup(nb_inh_units, name="inh"))
constraints1 = MinMaxConstraint(0.0, 1.0)


lplcon = model.add_connection(
    LPLConnection(
    input_group, 
    exc_neurons, 
    connection_prob=lpl_con_prob, 
    initial_weight=lpl_con_w, 
    name="LPLCon", 
    target="exc", 
    tau_post_mean=600,
    tau_var=30,
    constraints=constraints1,
    lamda_=1.0,
    phi=1.0,
    lr=1e-2,
    delta_time=20e-3,
    tau_rms=100.0,
    timestep_rmsprop_updates=timestep_rmsprop_updates))

conei = model.add_connection(
    Connection(
        exc_neurons, 
        inh_neurons, 
        gaussian=True,
        initial_weight=ei_con_w,  
        name="Conei", 
        target="exc"))

conii = model.add_connection(
    Connection(
        inh_neurons, 
        inh_neurons, 
        connection_prob=ii_con_prob, 
        initial_weight=ii_con_w, 
        name="Conii", 
        target="inh"))

conie = model.add_connection(
    Connection(
        inh_neurons, 
        exc_neurons, 
        gaussian=True,
        initial_weight=0.5, 
        name="Conie", 
        target="inh"))

generator = SparseGenerator(nb_workers=0)
model.configure(input=input_group, generator=generator, time_step=dt)
model.summary()

exc_neurons.random_mem(mean=-60e-3, sigma=10e-3, seed=42)
lplcon.set_evolve(True)

In [ ]:
model.stimulate(dataset1, epoch=epoch)

In [ ]:
list_neurons = [input_group.name, exc_neurons.name, inh_neurons.name]
list_con = [lplcon.name]
model.configure_stored(list_neurons, list_con, file_name + "Inhib_fixed")
model.stimulate(dataset2)